I feel I have exhausted my efforts with random forests(which is not true), so here I will try various implentation of gradient Boosting trees from sklearn, XGBoost, LightGBM with differet feature selection measures, include feature interactions, deal with outliers and may be finally if it requires we can stack models. 

In [1]:
!pip install tqdm

    100% |████████████████████████████████| 61kB 3.0MB/s 


In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from tqdm import tqdm
%matplotlib inline
rs = 100

In [0]:
train = pd.read_csv('drive/workspace/data/merc/train.csv')
test = pd.read_csv('drive/workspace/data/merc/test.csv')

In [0]:
cat_feat = list(train.select_dtypes(include=['object']).columns)
num_feat = list(train.select_dtypes(include=[np.number]).drop(['y'], axis=1).columns)

In [5]:
len(cat_feat), len(num_feat)

(8, 369)

So here cat_feat contains all categorical features, and num_feat has all numerical features excluding the target y. 

## Utility functions

Some of the utility functions are same as from nb2 but still here I am writing them again. 

In [0]:
def val_r2_score(estimator, X, y, n_splits=5, n_repeats=10, random_state=rs):
  val_scores = np.array([])
  trn_scores = np.array([])
  for i in tqdm(range(n_repeats)):
    kf = KFold(n_splits, True, random_state=rs+i)
    for ti, vi in kf.split(X):
      X_trn, X_val = X.iloc[ti], X.iloc[vi]
      y_trn, y_val = y.iloc[ti], y.iloc[vi]
      estimator.fit(X_trn, y_trn)
      val_scores = np.append(val_scores, r2_score(y_val, estimator.predict(X_val)))
      trn_scores = np.append(trn_scores, r2_score(y_trn, estimator.predict(X_trn)))
  return (val_scores.reshape((n_repeats, n_splits)), trn_scores.reshape( n_repeats, n_splits))

In [0]:
def me_val_r2_score(estimator, train, columns, n_splits=5, n_repeats=2, random_state=rs):
  val_scores = np.array([])
  for i in tqdm(range(n_repeats)) : 
    kf1 = KFold(n_splits=n_splits, shuffle=True, random_state=rs+i)
    for ti, vi in kf1.split(train):
      ttrain, vtrain = train.iloc[ti], train.iloc[vi]
      for col in columns: 
        means = ttrain.groupby(col)['y'].mean()
        ttrain[col+'_mt'] = ttrain[col].map(means)
        vtrain[col+'_mt'] = vtrain[col].map(means)
      
      vtrain.fillna(ttrain['y'].mean(), inplace=True)
      X_trn = ttrain
      X_val = vtrain
      
      kf2 = KFold(n_splits=5, shuffle=True, random_state=rs)
      for tind, vind in kf2.split(ttrain) : 
        ttrn, vtrn = ttrain.iloc[tind], ttrain.iloc[vind]
        for col in columns : 
          means = ttrn.groupby(col)['y'].mean()
          vtrn[col+'_mt'] = vtrn[col].map(means)
        X_trn.iloc[vind] = vtrn

      X_trn.fillna(ttrain['y'].mean(), inplace=True)
      
      _=estimator.fit(X_trn.drop(['y']+columns, axis=1), X_trn['y'])
      val_scores = np.append(val_scores, r2_score(X_val['y'], estimator.predict(X_val.drop(['y']+columns, axis=1))))
  return val_scores.reshape(n_repeats, n_splits)


In [0]:
# This function return a list of mean encoded validation and train dataframes to be used in me_r2_score function
# to calculate r2_scores for each. 
def me_df(train, n_splits=4, n_repeats=10, random_state=rs):
  trns = []
  vals = []
  for i in tqdm(range(n_repeats)):
    kf1 = KFold(n_splits, True, random_state=rs+i)
    for ti, vi in kf1.split(train):
      ttrain, vtrain = train.iloc[ti], train.iloc[vi]
      for col in cat_feat:
        means = ttrain.groupby(col)['y'].mean()
        ttrain[col+'_mt'] = ttrain[col].map(means)
        vtrain[col+'_mt'] = vtrain[col].map(means)
      vtrain.fillna(ttrain['y'].mean(), inplace=True)
      
      X_trn = ttrain.copy()
      X_val = vtrain.copy()
      
      kf2 = KFold(5, True, random_state=rs)
      for tind, vind in kf2.split(ttrain):
        ttrn, vtrn = ttrain.iloc[tind], ttrain.iloc[vind]
        for col in cat_feat:
          means = ttrn.groupby(col)['y'].mean()
          vtrn[col+'_mt'] = vtrn[col].map(means)
        X_trn.iloc[vind] = vtrn
      X_trn.fillna(ttrain['y'].mean(), inplace=True)
      trns.append(X_trn)
      vals.append(X_val)
  return trns, vals


def me_r2_score(estimator, trns, vals, me_cols, drop_feat=[], random_state=rs):
  val_scores=[]
  ohe_cols = [col for col in cat_feat if col not in me_cols]
  for trn, val in tqdm(zip(trns, vals)) : 
    trn = trn.drop(me_cols, axis=1)
    val = val.drop(me_cols, axis=1)
    for col in ohe_cols : 
      trn = trn.drop([col+'_mt'], axis=1)
      val = val.drop([col+'_mt'], axis=1)
    if len(ohe_cols) : 
      trn = pd.get_dummies(trn, columns=ohe_cols)
      val = pd.get_dummies(val, columns=ohe_cols)
        
    if len(drop_feat) :
      trn = trn.drop([f for f in drop_feat if f in list(trn.columns)], axis=1)
      val = val.drop([f for f in drop_feat if f in list(val.columns)], axis=1)
    
    _=estimator.fit(trn.drop(['y'],  axis=1), trn['y'])
    preds = estimator.predict(val.drop(['y'], axis=1))
    
    val_scores.append(r2_score(val['y'], preds))
  return val_scores
    


In [0]:
def validation_curve(estimator, trn, val, me_cols=cat_feat, drop_feat=[]):
    
    ohe_cols = [c for c in cat_feat if c not in me_cols]
    trn = trn.drop(me_cols, axis=1)
    val = val.drop(me_cols, axis=1)
    for col in ohe_cols : 
      trn = trn.drop([col+'_mt'], axis=1)
      val = val.drop([col+'_mt'], axis=1)
    if len(ohe_cols) : 
      trn = pd.get_dummies(trn, columns=ohe_cols)
      val = pd.get_dummies(val, columns=ohe_cols)
        
    if len(drop_feat) :
      trn = trn.drop([f for f in drop_feat if f in list(trn.columns)], axis=1)
      val = val.drop([f for f in drop_feat if f in list(val.columns)], axis=1)
    
    X_train = trn.drop(['y'], axis=1)
    X_val = val.drop(['y'], axis=1)
    y_train = trn['y']
    y_val = val['y']
    
    _ = estimator.fit(X_train, y_train)
    
    test_score = np.empty(len(estimator.estimators_))
    train_score = np.empty(len(estimator.estimators_))
    
    for i, pred in enumerate(estimator.staged_predict(X_val)):
        test_score[i] = r2_score(y_val, pred)

    for i, pred in enumerate(estimator.staged_predict(X_train)):
        train_score[i] = r2_score(y_train, pred)

    best_iter = np.argmax(test_score)
    
#     Uncommment the following lines to see a plot. 
    
    
#     test_line = plt.plot(test_score, label='test')

#     colour = test_line[-1].get_color()
#     plt.plot(train_score, '--', color=colour, label='train')

#     plt.xlabel("Number of boosting iterations")
#     plt.ylabel("R2 score")
#     plt.legend(loc='best')
#     plt.axvline(x=best_iter, color=colour)
    return best_iter

## Feature Selection 

First let's try to mark out features which have little variance ie which have fews ones. 

In [0]:
low_var_feat = ['X4', 'X4_mt']
var_df = pd.DataFrame([train[col].sum() for col in num_feat], index=num_feat, columns=[ 'ones'])
var_df.sort_values(by='ones', inplace=True)

In [11]:
var_df[var_df['ones']<50].shape
low_var_feat.extend(list(var_df[var_df['ones']<50].index))
len(low_var_feat)

(153, 1)

155

So there are 153 such columns which have less than 50 ones out of 4209. So we added them to drop list. Next we check if there are some duplicates. 

In [0]:
d_trn= train.T.drop_duplicates().T
dup_feat = [f for f in train if f not in d_trn]

In [13]:
print (len(dup_feat))

56


## Scikit Learn

Let's start with sklearn's implementation of GBM. 

In [0]:
X = train.drop(['y'], axis=1)
X = pd.get_dummies(X, columns=cat_feat)
y = train['y']

In [84]:
gbr = GradientBoostingRegressor()
val_scores, trn_scores = train_val_r2_score(gbr, X, y)

100%|██████████| 10/10 [01:20<00:00,  8.07s/it]


In [55]:
print(val_scores.mean(), val_scores.std())
print (trn_scores.mean(), trn_scores.std())
print (val_scores)

(0.5592712508115831, 0.05244097706310926)
(0.6577529734988582, 0.010593274705071789)
[[0.52770343 0.50505212 0.5545412  0.6141021  0.56593822]
 [0.60448107 0.61669633 0.51661907 0.5503775  0.47292347]
 [0.6047506  0.60202299 0.57138764 0.54706123 0.47375107]
 [0.5818624  0.43133665 0.58951768 0.56070441 0.60799904]
 [0.58047576 0.61070046 0.59302227 0.61685635 0.44035624]
 [0.55522127 0.58194286 0.62588408 0.5726197  0.47735789]
 [0.60163942 0.47447391 0.63212972 0.57288287 0.50519909]
 [0.52208136 0.57322103 0.5623261  0.51713715 0.60830957]
 [0.45906274 0.53498321 0.59982736 0.63724457 0.59071208]
 [0.58256642 0.5470017  0.61531224 0.49681789 0.577369  ]]


I will train a full vanilla GBM model and submit to see how it fares on leaderboard compared our score here. 

In [65]:
X_test = pd.get_dummies(test, columns=cat_feat)
X_test.shape, X.shape

((4209, 570), (4209, 564))

So there are some columns(ie some level of categorical columns) that are in test in not in train. So for the columns only in train we add them to test and for columns only in test we drop them. 

In [66]:
col1 = [f for f in X_test.columns if f not in X.columns]
col2 = [f for f in X.columns if f not in X_test.columns]
len(col1), len(col2)

(16, 10)

In [67]:
# We drop col1 from X_test and add col2 to X_test
X_test = X_test.drop(col1, axis=1)
new_df = pd.DataFrame(np.zeros((X_test.shape[0],len(col2))), index=X_test.index, columns=col2)
X_test = pd.concat([X_test, new_df], axis=1)
X_test.shape, X.shape

((4209, 564), (4209, 564))

In [70]:
gbr = GradientBoostingRegressor()
gbr.fit(X,y)
y_pred = gbr.predict(X_test)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [0]:
subm = np.stack((X_test.ID, y_pred), axis=1)
np.savetxt('drive/workspace/data/merc/subm18.csv', subm, delimiter=',', header='ID,y', comments='', fmt=['%d','%.6f'])

Public LB : 0.52614, PrivateLB : 0.52965

Next instead of one hot encoding, we will target_encode our categories and then tune the GBM parameters. 

So first we get a list of trains and validation dataframes from multiple repeats of KFold in mean-encoded form, so that we don't have to to geenrate them everytime we test a new model with changed parameters. 


In [0]:
n_splits=4
n_repeats=10
trns, vals = me_df(train, n_splits, n_repeats)

In [150]:
gbr = GradientBoostingRegressor()
val_scores = np.array(me_r2_score(gbr, trns, vals, cat_feat)).reshape((n_repeats, n_splits))

100%|██████████| 50/50 [02:04<00:00,  2.49s/it]


In [160]:
print (val_scores.mean(), val_scores.std())

(0.5648632147848137, 0.055460594449785315)


Let's tune our gbr model. 

First we will figure out the n_estimators for a sufficiently high learning rate. For this n_estimators and learning rate we tune other parameters and later we can lower learning rate and increase n_estimators. 

In [170]:
trns, vals = me_df(train)

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [0]:
params = dict(n_estimators=500, max_depth=5, max_features='sqrt', learning_rate=0.1, subsample=.8, random_state=rs)
gbr = GradientBoostingRegressor(**params)

In [203]:
# validation_curve(gbr, trns[1], vals[1])
best_iters = []
for trn, val in tqdm(zip(trns, vals)): 
  best_iters.append(validation_curve(gbr, trn, val))

100%|██████████| 40/40 [01:41<00:00,  2.54s/it]


In [204]:
print (np.array(best_iters).mean(), np.array(best_iters).std())

(52.85, 13.047126120337767)


So we have n_estimators=53 where we have highest val_score for learning rate=0.1 . So we will stick with that and tune other params. Later we will also  delve into feature selection. Before we tune let's see if we improve scores by dropping near zero variance features. So we drop features from dup_feat and low_var_feat. 



In [262]:
n_splits=4
n_repeats=10
trns, vals = me_df(train, n_splits, n_repeats)

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [0]:
gbr = GradientBoostingRegressor(random_state=rs)
val_scores = np.array(me_r2_score(gbr, trns, vals, cat_feat, drop_feat=list(set(low_var_feat+dup_feat)))).reshape((n_repeats, n_splits))

In [279]:
print (val_scores.mean(), val_scores.std())

(0.5651195117270597, 0.049668760519681034)


Just miniscule improvement in score. Again let's find an optimal value for n_estimators. 

In [0]:
params = dict(n_estimators=300, random_state=rs)
gbr = GradientBoostingRegressor(**params)

In [0]:
best_iters = []
for trn, val in tqdm(zip(trns, vals)): 
  best_iters.append(validation_curve(gbr, trn, val, drop_feat=list(set(low_var_feat+dup_feat))))

In [285]:
print (np.array(best_iters).mean(), np.array(best_iters).std())

(49.45, 20.179135263930416)


Too high deviation. Let's take optimal value to be 70 so that we don't underfit and find scores. 

In [0]:
gbr = GradientBoostingRegressor(n_estimators=70, random_state=rs)
val_scores = np.array(me_r2_score(gbr, trns, vals, cat_feat, drop_feat=list(set(low_var_feat+dup_feat)))).reshape((n_repeats, n_splits))

In [288]:
print (val_scores.mean(), val_scores.std())

(0.5678109876250517, 0.049773153219372396)


No doubt there is some improvement, but this high variation is bugging me. Anyway let's submit this and see. 

In [0]:
me_cols = cat_feat
train_new = train.copy()
test_new = test.copy()
for col in me_cols : 
  means = train_new.groupby(col)['y'].mean()
  train_new[col+'_mt'] = train_new[col].map(means)
  test_new[col+'_mt'] = test_new[col].map(means)

test_new.fillna(train_new['y'].mean(), inplace=True)
 

In [299]:
# let's regularise train_new

train_new1 = train_new.copy()

kf = KFold(n_splits=5, shuffle=True, random_state=rs)
for ti, vi in kf.split(train_new):
  trn, val = train_new.iloc[ti], train_new.iloc[vi]
  for col in me_cols:
    means = trn.groupby(col)['y'].mean()
    val[col+'_mt'] = val[col].map(means)
  train_new1.iloc[vi] = val
train_new1.fillna(train_new['y'].mean(), inplace=True)
    

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [0]:
feat_to_drop = list(set(['y'] + cat_feat + dup_feat + low_var_feat))
X_trn = train_new1.drop(feat_to_drop, axis=1)
y_trn = train_new1['y']
X_test = test_new[X_trn.columns]

In [306]:
gbr = GradientBoostingRegressor(n_estimators=70, random_state=rs)
gbr.fit(X_trn, y_trn)
y_pred = gbr.predict(X_test)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=70, presort='auto', random_state=100,
             subsample=1.0, verbose=0, warm_start=False)

In [0]:
subm = np.stack((test.ID, y_pred), axis=1)
np.savetxt('drive/workspace/data/merc/subm19.csv', subm, delimiter=',', comments='', header='ID,y', fmt=['%d', '%.6f'])

Private LB : 0.54652  Public LB : 0.55235 . Okay decent score, and may be we will improve as we tune this model. 

#### max_depth

So a defaut max_depth of 3 was used previously. 

In [0]:
trns, vals = me_df(train)

In [0]:
max_depth = [2,5]
val_scores = np.empty(shape=(len(max_depth),len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in range(len(max_depth)) : 
  gbr = GradientBoostingRegressor(n_estimators=70, random_state=rs, max_depth=max_depth[i])
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat=drop_feat)

In [22]:
val_scores.mean(axis=1)
val_scores.std(axis=1)

array([0.571426  , 0.54834443])

array([0.05009221, 0.0518472 ])

In [0]:
gbr = GradientBoostingRegressor(n_estimators=70, max_depth=1, random_state=rs)
val_scores = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)

In [25]:
np.array(val_scores).mean(), np.array(val_scores).std()

(0.5693752833603517, 0.048950054587181664)

Looks like max_depth=2 gives the highest r2_score though with a high variation. Next we will tune min_samples_split. 

#### min_samples_split

In [0]:
min_samples_split = [5,8,12]
val_scores = np.empty(shape=(len(min_samples_split), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(min_samples_split))):
  gbr = GradientBoostingRegressor(n_estimators=70, max_depth=2, random_state=rs, min_samples_split=min_samples_split[i])
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)

In [37]:
val_scores.mean(axis=1), val_scores.std(axis=1)

(array([0.57148897, 0.57156314, 0.57155459]),
 array([0.05005749, 0.05005552, 0.05005265]))

In [0]:
min_samples_split = [15,20,25]
val_scores = np.empty(shape=(len(min_samples_split), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(min_samples_split))):
  gbr = GradientBoostingRegressor(n_estimators=70, max_depth=2, random_state=rs, min_samples_split=min_samples_split[i])
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)

In [40]:
val_scores.mean(axis=1), val_scores.std(axis=1)

(array([0.57155459, 0.57156079, 0.57156079]),
 array([0.05005265, 0.05005868, 0.05005868]))

In [0]:
min_samples_split = [30, 35, 40]
val_scores = np.empty(shape=(len(min_samples_split), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(min_samples_split))):
  gbr = GradientBoostingRegressor(n_estimators=70, max_depth=2, random_state=rs, min_samples_split=min_samples_split[i])
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)


In [42]:
val_scores.mean(axis=1), val_scores.std(axis=1)

(array([0.57155951, 0.57155797, 0.57169769]),
 array([0.05005879, 0.05005591, 0.05030246]))

In [0]:
min_samples_split = [45, 55, 70]
val_scores = np.empty(shape=(len(min_samples_split), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(min_samples_split))):
  gbr = GradientBoostingRegressor(n_estimators=70, max_depth=2, random_state=rs, min_samples_split=min_samples_split[i])
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)


In [45]:
val_scores.mean(axis=1), val_scores.std(axis=1)

(array([0.57199388, 0.57215282, 0.57215772]),
 array([0.05011747, 0.04996139, 0.04996028]))

In [0]:
min_samples_split = [80, 100, 130]
val_scores = np.empty(shape=(len(min_samples_split), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(min_samples_split))):
  gbr = GradientBoostingRegressor(n_estimators=70, max_depth=2, random_state=rs, min_samples_split=min_samples_split[i])
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)


In [47]:
val_scores.mean(axis=1), val_scores.std(axis=1)

(array([0.57229248, 0.57231745, 0.57219802]),
 array([0.05001029, 0.04991818, 0.04998081]))

In [0]:
min_samples_split = [120, 135, 150]
val_scores = np.empty(shape=(len(min_samples_split), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(min_samples_split))):
  gbr = GradientBoostingRegressor(n_estimators=70, max_depth=2, random_state=rs, min_samples_split=min_samples_split[i])
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)


In [49]:
val_scores.mean(axis=1), val_scores.std(axis=1)

(array([0.57223828, 0.57216949, 0.57219447]),
 array([0.04994356, 0.05002337, 0.04994892]))

In [0]:
min_samples_split = [170, 200]
val_scores = np.empty(shape=(len(min_samples_split), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(min_samples_split))):
  gbr = GradientBoostingRegressor(n_estimators=70, max_depth=2, random_state=rs, min_samples_split=min_samples_split[i])
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)


In [51]:
val_scores.mean(axis=1), val_scores.std(axis=1)

(array([0.57231448, 0.57238109]), array([0.04982459, 0.04970026]))

In [0]:
min_samples_split = [250, 300, 400]
val_scores = np.empty(shape=(len(min_samples_split), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(min_samples_split))):
  gbr = GradientBoostingRegressor(n_estimators=70, max_depth=2, random_state=rs, min_samples_split=min_samples_split[i])
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)


In [53]:
val_scores.mean(axis=1), val_scores.std(axis=1)

(array([0.57229568, 0.57228083, 0.5723353 ]),
 array([0.04968895, 0.04962757, 0.04970672]))

In [0]:
min_samples_split = [500, 650, 800]
val_scores = np.empty(shape=(len(min_samples_split), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(min_samples_split))):
  gbr = GradientBoostingRegressor(n_estimators=70, max_depth=2, random_state=rs, min_samples_split=min_samples_split[i])
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)


In [55]:
val_scores.mean(axis=1), val_scores.std(axis=1)

(array([0.57236812, 0.5724001 , 0.57259389]),
 array([0.04970903, 0.04963461, 0.04955193]))

In [0]:
min_samples_split = [900, 1000, 1200]
val_scores = np.empty(shape=(len(min_samples_split), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(min_samples_split))):
  gbr = GradientBoostingRegressor(n_estimators=70, max_depth=2, random_state=rs, min_samples_split=min_samples_split[i])
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)


In [57]:
val_scores.mean(axis=1), val_scores.std(axis=1)

(array([0.57260135, 0.57279383, 0.57286719]),
 array([0.04966826, 0.04963141, 0.04945235]))

In [0]:
min_samples_split = [1500, 1800, 2200]
val_scores = np.empty(shape=(len(min_samples_split), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(min_samples_split))):
  gbr = GradientBoostingRegressor(n_estimators=70, max_depth=2, random_state=rs, min_samples_split=min_samples_split[i])
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)


In [59]:
val_scores.mean(axis=1), val_scores.std(axis=1)

(array([0.5735213 , 0.57281518, 0.57286785]),
 array([0.0498146 , 0.04968227, 0.04968578]))

This makes some sense now, since our max_depth is 2 so score starts decreasing at ver high values of min_sample_split. Anyway let's look deeper. 

In [0]:
min_samples_split = [1300, 1400, 1600]
val_scores = np.empty(shape=(len(min_samples_split), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(min_samples_split))):
  gbr = GradientBoostingRegressor(n_estimators=70, max_depth=2, random_state=rs, min_samples_split=min_samples_split[i])
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)


In [61]:
val_scores.mean(axis=1), val_scores.std(axis=1)

(array([0.5729889 , 0.57282316, 0.57350778]),
 array([0.0494858 , 0.04954945, 0.04979938]))

In [0]:
min_samples_split = [1450, 1550]
val_scores = np.empty(shape=(len(min_samples_split), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(min_samples_split))):
  gbr = GradientBoostingRegressor(n_estimators=70, max_depth=2, random_state=rs, min_samples_split=min_samples_split[i])
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)


In [63]:
val_scores.mean(axis=1), val_scores.std(axis=1)

(array([0.57296228, 0.57350565]), array([0.04979288, 0.04979993]))

In [64]:
min_samples_split = [1520, 1550, 2500]
val_scores = np.empty(shape=(len(min_samples_split), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(min_samples_split))):
  gbr = GradientBoostingRegressor(n_estimators=70, max_depth=2, random_state=rs, min_samples_split=min_samples_split[i])
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)

  


100%|██████████| 3/3 [00:50<00:00, 16.95s/it]


In [65]:
val_scores.mean(axis=1), val_scores.std(axis=1)

(array([0.5735213 , 0.57350565, 0.57295383]),
 array([0.0498146 , 0.04979993, 0.04957867]))

So min_samples_split=1500 seems to be the optimal value. Let's move on to min_samples_leaf. 

#### min_samples_leaf

In [0]:
min_samples_leaf = [50, 100, 200]
val_scores = np.empty(shape=(len(min_samples_split), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(min_samples_leaf))):
  gbr = GradientBoostingRegressor(n_estimators=70, max_depth=2, random_state=rs, min_samples_split=1500, min_samples_leaf=min_samples_leaf[i])
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)


In [67]:
val_scores.mean(axis=1), val_scores.std(axis=1)

(array([0.56792298, 0.56815489, 0.55831368]),
 array([0.05004255, 0.04981119, 0.04874999]))

In [0]:
min_samples_leaf = [5, 10]
val_scores = np.empty(shape=(len(min_samples_leaf), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(min_samples_leaf))):
  gbr = GradientBoostingRegressor(n_estimators=70, max_depth=2, random_state=rs, min_samples_split=1500, min_samples_leaf=min_samples_leaf[i])
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)


In [72]:
val_scores.mean(axis=1), val_scores.std(axis=1)

(array([0.57209076, 0.57244787]), array([0.04989728, 0.04989285]))

In [0]:
min_samples_leaf = [15, 22, 30]
val_scores = np.empty(shape=(len(min_samples_leaf), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(min_samples_leaf))):
  gbr = GradientBoostingRegressor(n_estimators=70, max_depth=2, random_state=rs, min_samples_split=1500, min_samples_leaf=min_samples_leaf[i])
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)


In [74]:
val_scores.mean(axis=1), val_scores.std(axis=1)

(array([0.57247223, 0.57214157, 0.57215968]),
 array([0.04982567, 0.04981692, 0.04985344]))

In [0]:
min_samples_leaf = [3, 12, 14]
val_scores = np.empty(shape=(len(min_samples_leaf), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(min_samples_leaf))):
  gbr = GradientBoostingRegressor(n_estimators=70, max_depth=2, random_state=rs, min_samples_split=1500, min_samples_leaf=min_samples_leaf[i])
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)


In [77]:
val_scores.mean(axis=1), val_scores.std(axis=1)

(array([0.57191485, 0.57251959, 0.57250211]),
 array([0.04992553, 0.0498513 , 0.0499633 ]))

So min_sample_leaf=1 gives the best score. Let's now tune subsample. 

#### subsample

In [0]:
subsample=[0.75, 0.8, 0.85, 0.9, 0.95]
val_scores = np.empty(shape=(len(subsample), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(subsample))):
  gbr = GradientBoostingRegressor(n_estimators=70, max_depth=2, random_state=rs, min_samples_split=1500, subsample=subsample[i])
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)


In [84]:
val_scores.mean(axis=1), val_scores.std(axis=1)

(array([0.57240977, 0.57249681, 0.57264502, 0.57335653, 0.57316341]),
 array([0.04997876, 0.0496694 , 0.04979318, 0.04998372, 0.04978146]))

In [0]:
subsample=[0.88, 0.92, 9.94, 0.96]
val_scores = np.empty(shape=(len(subsample), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(subsample))):
  gbr = GradientBoostingRegressor(n_estimators=70, max_depth=2, random_state=rs, min_samples_split=1500, subsample=subsample[i])
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)


In [0]:
val_scores.mean(axis=1), val_scores.std(axis=1)

(array([0.57240977, 0.57249681, 0.57264502, 0.57335653, 0.57316341]),
 array([0.04997876, 0.0496694 , 0.04979318, 0.04998372, 0.04978146]))

In [0]:
subsample=[0.93, 0.97]
val_scores = np.empty(shape=(len(subsample), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(subsample))):
  gbr = GradientBoostingRegressor(n_estimators=70, max_depth=2, random_state=rs, min_samples_split=1500, subsample=subsample[i])
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)


In [87]:
val_scores.mean(axis=1), val_scores.std(axis=1)

(array([0.5728964 , 0.57317337]), array([0.04974105, 0.04996316]))

subsample=1. Let's now decrease learning rate and increase n_estimators. 

#### learning_rate

In [92]:
val_scores = []
gbr = GradientBoostingRegressor(n_estimators=140, learning_rate=0.05, max_depth=2, random_state=rs, min_samples_split=1500)
val_scores = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)

100%|██████████| 40/40 [00:35<00:00,  1.13it/s]


In [93]:
np.array(val_scores).mean()

0.5736421978487625

In [94]:
val_scores = []
gbr = GradientBoostingRegressor(n_estimators=700, learning_rate=0.01, max_depth=2, random_state=rs, min_samples_split=1500)
val_scores = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)

100%|██████████| 40/40 [02:43<00:00,  4.10s/it]


In [95]:
np.array(val_scores).mean()

0.5735211126899441

In [96]:
val_scores = []
gbr = GradientBoostingRegressor(n_estimators=350, learning_rate=0.02, max_depth=2, random_state=rs, min_samples_split=1500)
val_scores = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)

100%|██████████| 40/40 [01:22<00:00,  2.06s/it]


In [97]:
np.array(val_scores).mean()

0.5735218137559037

In [98]:
val_scores = []
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.07, max_depth=2, random_state=rs, min_samples_split=1500)
val_scores = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)

100%|██████████| 40/40 [00:25<00:00,  1.59it/s]


In [99]:

np.array(val_scores).mean()

0.5736051320599398

In [100]:
val_scores = []
gbr = GradientBoostingRegressor(n_estimators=175, learning_rate=0.04, max_depth=2, random_state=rs, min_samples_split=1500)
val_scores = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)

100%|██████████| 40/40 [00:41<00:00,  1.04s/it]


In [101]:
np.array(val_scores).mean()

0.5735349280745389

In [102]:
val_scores = []
gbr = GradientBoostingRegressor(n_estimators=117, learning_rate=0.06, max_depth=2, random_state=rs, min_samples_split=1500)
val_scores = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)

100%|██████████| 40/40 [00:27<00:00,  1.43it/s]


In [103]:
np.array(val_scores).mean()

0.5735442526301707

So lr=0.05. Since now our parameters are changes let's again find out the optimal n_estimators for the new tuned parameters using validation_curve function. 

In [104]:
gbr = GradientBoostingRegressor(n_estimators=500, learning_rate=0.05, max_depth=2, random_state=rs, min_samples_split=1500)
n_estimators=[]
for trn, val in tqdm(zip(trns, vals)):
  n_estimators.append(validation_curve(gbr, trn, val, drop_feat=drop_feat))


100%|██████████| 40/40 [02:09<00:00,  3.24s/it]


In [107]:
np.array(n_estimators).mean(), np.array(n_estimators).std()

(238.05, 126.49168945033504)

See our n_estimators have increased, let's check score for other values of n_estimators. 

In [0]:
n_estimators=[140, 240, 300, 350, 400]
val_scores = np.empty(shape=(len(n_estimators), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(n_estimators))):
  gbr = GradientBoostingRegressor(n_estimators=n_estimators[i], learning_rate=0.05, max_depth=2, random_state=rs, min_samples_split=1500)
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)


In [115]:
val_scores.mean(axis=1),val_scores.std(axis=1)

(array([0.5736422 , 0.57342363, 0.573095  , 0.57273635, 0.57205834]),
 array([0.04961387, 0.04969295, 0.04982322, 0.05001055, 0.05005102]))

In [0]:
n_estimators=[120, 170]
val_scores = np.empty(shape=(len(n_estimators), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(n_estimators))):
  gbr = GradientBoostingRegressor(n_estimators=n_estimators[i], learning_rate=0.05, max_depth=2, random_state=rs, min_samples_split=1500)
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)


In [117]:
val_scores.mean(axis=1),val_scores.std(axis=1)

(array([0.57329583, 0.57365369]), array([0.04956578, 0.0496248 ]))

In [0]:
n_estimators=[155, 165]
val_scores = np.empty(shape=(len(n_estimators), len(trns)))
drop_feat = list(set(low_var_feat+dup_feat))
for i in tqdm(range(len(n_estimators))):
  gbr = GradientBoostingRegressor(n_estimators=n_estimators[i], learning_rate=0.05, max_depth=2, random_state=rs, min_samples_split=1500)
  val_scores[i] = me_r2_score(gbr, trns, vals, cat_feat, drop_feat)


In [121]:
val_scores.mean(axis=1),val_scores.std(axis=1)

(array([0.57368584, 0.57366262]), array([0.04964195, 0.04961507]))

Looks like optimal value of n_estimators=155 for learning_rate=0.05 . 

Finally let's submit with following parameters : 
1. n_estimators=140
2. learning_rate=0.05
3. max_depth=2
4. min_sample_split=1500

In [0]:
me_cols = cat_feat
train_new = train.copy()
test_new = test.copy()
for col in me_cols : 
  means = train_new.groupby(col)['y'].mean()
  train_new[col+'_mt'] = train_new[col].map(means)
  test_new[col+'_mt'] = test_new[col].map(means)

test_new.fillna(train_new['y'].mean(), inplace=True)
 

In [123]:
# let's regularise train_new

train_new1 = train_new.copy()

kf = KFold(n_splits=5, shuffle=True, random_state=rs)
for ti, vi in kf.split(train_new):
  trn, val = train_new.iloc[ti], train_new.iloc[vi]
  for col in me_cols:
    means = trn.groupby(col)['y'].mean()
    val[col+'_mt'] = val[col].map(means)
  train_new1.iloc[vi] = val
train_new1.fillna(train_new['y'].mean(), inplace=True)
    

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [0]:
feat_to_drop = list(set(['y'] + cat_feat + dup_feat + low_var_feat))
X_trn = train_new1.drop(feat_to_drop, axis=1)
y_trn = train_new1['y']
X_test = test_new[X_trn.columns]

In [125]:
gbr = GradientBoostingRegressor(n_estimators=155, learning_rate=0.05, max_depth=2, random_state=rs, min_samples_split=1500)
gbr.fit(X_trn, y_trn)
y_pred = gbr.predict(X_test)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.05, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=1500, min_weight_fraction_leaf=0.0,
             n_estimators=155, presort='auto', random_state=100,
             subsample=1.0, verbose=0, warm_start=False)

In [0]:
subm = np.stack((test.ID, y_pred), axis=1)
np.savetxt('drive/workspace/data/merc/subm20.csv', subm, delimiter=',', comments='', header='ID,y', fmt=['%d', '%.6f'])

Public LB : 0.55443, Private=0.54789. This is the best score till now. 